# Capítulo 06: Gestión del dinero

💰 Únete a la comunidad de [Discord](https://discord.gg/z3dx5XpkX4)

📚 Puedes leer nuestro libro en [Amazon](https://www.amazon.es/Python-para-finanzas-trading-algor%C3%ADtmico-ebook/dp/B0BT4ZS9Q3)

🖥️ El canal de [YouTube de Quantreo's](https://www.youtube.com/channel/UCp7jckfiEglNf_Gj62VR0pw) (en inglés) y el de [Frogames](https://www.youtube.com/channel/UCMUxXNYrVCv6-bQakhomvBg) en Español


In [1]:
# Importar las librerías
import MetaTrader5 as mt5
from datetime import datetime
import numpy as np

C:\Users\juangabriel\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\juangabriel\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\juangabriel\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [33]:
# Crear la conexión entre MT5 y Python
mt5.initialize()

True

### Colocar una orden con take profit & stop loss

In [36]:
# Colocar una orden de compra con take profit
# Valor de inicialización
lot = 0.1
symbol = "EURUSD"

# Extraemos el punto del símbolo
point = mt5.symbol_info(symbol).point

# Elegimos la desviación
deviation = 10

#Elegimos el filling model del símbolo
filling_type = mt5.symbol_info(symbol).filling_mode

# Creamos una petición usando el diccionario
request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": lot,
    "type": mt5.ORDER_TYPE_BUY,
    "price": mt5.symbol_info_tick(symbol).ask,
    "deviation": deviation,
    "tp": mt5.symbol_info_tick(symbol).ask + 100*point,
    "sl": mt5.symbol_info_tick(symbol).ask - 100*point, 
    "type_filling": 1, #filling_type,
    "type_time": mt5.ORDER_TIME_GTC,
}

mt5.order_check(request).comment

'Done'

In [38]:
# Colocar una orden de venta con take profit
# Valor de inicialización
lot = 0.1
symbol = "EURUSD"

# Extraemos el punto del símbolo
point = mt5.symbol_info(symbol).point

# Elegimos la desviación
deviation = 10

#Elegimos el filling model del símbolo
filling_type = 1#mt5.symbol_info(symbol).filling_mode

# Creamos una petición usando el diccionario
request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": lot,
    "type": mt5.ORDER_TYPE_SELL,
    "price": mt5.symbol_info_tick(symbol).bid,
    "deviation": deviation,
    "tp": mt5.symbol_info_tick(symbol).bid - 100 * point,
    "sl": mt5.symbol_info_tick(symbol).bid + 100 * point, 
    "type_filling": filling_type,
    "type_time": mt5.ORDER_TIME_GTC,
}

mt5.order_check(request).comment

'Done'

### Encontrar el stop loss y take profit para un porcentaje de riesgo específico

In [16]:
def risk_reward_threshold(symbol, buy=True, risk=0.01, reward=0.02):
    
    # Extraemos el apalancamiento
    leverage = mt5.account_info().leverage

    # Calculamos el precio de referencia
    price = mt5.symbol_info(symbol).ask

    # Extraemos el número de decimales
    nb_decimal = str(price)[::-1].find(".")


    # Calculamos las variaciones en porcentaje
    var_down = risk/leverage
    var_up = reward/leverage


    # Encontrar el TP y SL usando el precio absoluto
    if buy:
        price = mt5.symbol_info(symbol).ask

        # Calculamos las variaciones en valores absluto
        price_var_down = var_down*price
        price_var_up = var_up * price

        tp = np.round(price + price_var_up, nb_decimal)
        sl = np.round(price - price_var_down, nb_decimal)

    else:

        price = mt5.symbol_info(symbol).bid

        # Calculamos las variaciones en valores absluto
        price_var_down = var_down*price
        price_var_up = var_up * price

        tp = np.round(price - price_var_up, nb_decimal)
        sl = np.round(price + price_var_down, nb_decimal)


        
    print(f"PRICE: {price} \t Take Profit: {tp} \t Stop Loss: {sl}")

In [17]:
risk_reward_threshold("EURUSD",buy=True)

PRICE: 1.06871 	 Take Profit: 1.06892 	 Stop Loss: 1.0686


In [19]:
(1.06892 - 1.06871) / 1.06871 * 100 * 100

1.9649858240312477

### Encontrar el volumen en función de nuestro capital

In [28]:
def position_size(capital, symbol):
    mt5.initialize()
    print(f"CAPITAL INVERTIDO: {capital}")
    
    leverage = mt5.account_info().leverage
    print(f"APALANCAMIENTO: {leverage}")
    
    invested_capital = capital  * leverage
    print(f"CAPITAL INVERTIDO APALANCADO: {invested_capital}")
    
    trade_size = mt5.symbol_info(symbol).trade_contract_size
    print(f"TAMAÑO DEL COMERCIO: {trade_size}")
    
    price = (mt5.symbol_info(symbol).ask + mt5.symbol_info(symbol).bid)/2
    print(f"PRECIO: {price}")

    lot_size = invested_capital / trade_size / price
    print(f"TAMAÑO DEL LOTE: {lot_size}")
    
    min_lot = mt5.symbol_info(symbol).volume_min
    print(f"LOTE MÍNIMO: {min_lot}")
    
    max_lot = mt5.symbol_info(symbol).volume_max
    print(f"LOTE MÁXIMO: {max_lot}")


    if min_lot<lot_size:
        number_decimal = str(min_lot)[::-1].find(".")
        print(f"CIFRAS DECIMALES: {number_decimal}")

        if number_decimal>0:
            lot_size_rounded = np.round(lot_size, number_decimal)
            print(f"TAMAÑO DEL LOTE REDONDEADO: {lot_size_rounded}")

            if lot_size < lot_size_rounded:
                lot_size_rounded = np.round(lot_size_rounded - min_lot, number_decimal)
                print(f"TAMAÑO DEL LOTE TRUNCADO: {lot_size_rounded}")

        else:
            number_size_lot =  len(str(min_lot))

            lot_size_rounded = int(np.round(lot_size, -number_size_lot))

            if lot_size < lot_size_rounded:
                lot_size_rounded = int(np.round(lot_size_rounded - number_size_lot, - number_size_lot))
                
        if lot_size_rounded>max_lot:
            lot_size_rounded = max_lot
            
        print(f"TAMAÑO DEL LOTE SUGERIDO: {lot_size_rounded}")
        return lot_size_rounded
    else: 
        print("El capital invertido es demasiado pequeño para poder hacer un pedido")
        
    
    
position_size(110, "EURUSD")

CAPITAL INVERTIDO: 110
APALANCAMIENTO: 100
CAPITAL INVERTIDO APALANCADO: 11000
TAMAÑO DEL COMERCIO: 10000.0
PRECIO: 1.06796
TAMAÑO DEL LOTE: 1.0300011236375894
LOTE MÍNIMO: 0.1
LOTE MÁXIMO: 100.0
CIFRAS DECIMALES: 1
TAMAÑO DEL LOTE REDONDEADO: 1.0
TAMAÑO DEL LOTE SUGERIDO: 1.0


1.0